# 資料整理和匯入

In [1]:
#%%
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler
import joblib

import numpy as np
import pandas as pd
import os

# 設定LSTM往前看的筆數和預測筆數
LookBackNum = 12 # LSTM往前看的筆數
ForecastNum = 48 # 預測題目當天的筆數 9:00至16:59

# 載入訓練資料
# DataName = os.getcwd() + '/L2_0318_11.csv'
DataName = os.getcwd() + '/L1_0102_1.csv'
SourceData = pd.read_csv(DataName, encoding='utf-8')

# 選擇要留下來的資料欄位
# (風速,大氣壓力,溫度,濕度,光照度,發電量)
AllOutPut = SourceData[['WindSpeed(m/s)','Pressure(hpa)','Temperature(°C)','Humidity(%)','Sunlight(Lux)', 'Power(mW)']].values

#(風速,大氣壓力,溫度,濕度,光照度)
#(發電量)
# LSTM_X_train = SourceData[['WindSpeed(m/s)','Pressure(hpa)','Temperature(°C)','Humidity(%)','Sunlight(Lux)']].values
# LSTM_y_train = SourceData[['Power(mW)']].values


# 正規化
LSTM_MinMaxModel = MinMaxScaler().fit(AllOutPut)
AllOutPut_MinMax = LSTM_MinMaxModel.transform(AllOutPut)
X_train = []
y_train = []

# 設定每i-LookBackNum筆資料(X_train)對應到第i筆資料(y_train)
for i in range(LookBackNum, len(AllOutPut_MinMax)):
  # 每次取前 LookBackNum 筆資料作為 X_train
  X_train.append(AllOutPut_MinMax[i - LookBackNum:i, :])
  # 將第i筆資料作為 y_train
  y_train.append(AllOutPut_MinMax[i, :])

X_train = np.array(X_train)
y_train = np.array(y_train)

# Reshaping
#(samples 是訓練樣本數量,timesteps 是每個樣本的時間步長,features 是每個時間步的特徵數量)
X_train = np.reshape(X_train,(X_train.shape [0], X_train.shape [1], 6)) # 6個特徵，包含回歸的發電量

print(X_train.shape)

(60, 12, 6)


# 建置LSTM模型&訓練模型

In [4]:
#%%
#============================建置&訓練模型============================
#建置LSTM模型

regressor = Sequential ()

# LSTM 層，這裡的 input_shape 是 X_train 的形狀 (LookBackNum, 6)，因為每個時間步長包含6個特徵
regressor.add(LSTM(units=256, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))

# LSTM 層
regressor.add(LSTM(units=128))

# Dropout 層避免過擬合
regressor.add(Dropout(0.3))

# 輸出層，預測發電量
regressor.add(Dense(units=1))  # 這裡的 1 是輸出的發電量

# 編譯模型
learning_rate = 0.01
optimizer = Adam(learning_rate=learning_rate)
regressor.compile(optimizer=optimizer, loss='mean_squared_error')

#開始訓練
regressor.fit(X_train, y_train, epochs = 300, batch_size = 32)

#保存模型
regressor.save('WheatherLSTM_Model.h5')
print('Model Saved')


Epoch 1/300


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 62ms/step - loss: 2.9893
Epoch 2/300
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - loss: 0.4908
Epoch 3/300
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - loss: 0.3959
Epoch 4/300
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - loss: 0.3108
Epoch 5/300
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step - loss: 0.1347
Epoch 6/300
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step - loss: 0.1389
Epoch 7/300
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step - loss: 0.1032
Epoch 8/300
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - loss: 0.1010
Epoch 9/300
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - loss: 0.1061
Epoch 10/300
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step - loss: 0.0994
Epoch 11/300
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - loss: 0.0983
Epoch 12/300
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - loss: 0.1016
Epoch 13/300
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - loss: 0.1004
Epoch 14/300
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - loss: 0.0957
Epoch 15/300
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step - loss: 0.0940
Epoch 16/300
2/2 ━━━━━━━━━━━━━━

Model Saved


# 預測數據

In [5]:
# %%
#============================預測數據============================

# 載入 LSTM 模型
regressor = load_model('WheatherLSTM_Model.h5')

# 初始化輸入資料和預測結果
inputs = []  # 存放參考資料
PredictOutput = []  # 存放預測值

# 取輸入資料的最後 LookBackNum 筆資料，包括發電量（共 6 個特徵）
TempData = AllOutPut[-LookBackNum:].reshape(LookBackNum, 6)  # 取前 6 個特徵，包括發電量
print("TempData.shape:", TempData.shape)

# 正規化資料，使用訓練時所使用的 LSTM_MinMaxModel
TempData = LSTM_MinMaxModel.transform(TempData)  # 正規化
print("TempData after scaling:", TempData.shape)

# 初始化輸入資料
inputs = [TempData]  # 初始化 inputs，形狀為 (1, 12, 6)，即 (batch_size, timesteps, features)
print("inputs.shape:", np.array(inputs).shape)

# 預測迴圈
for i in range(ForecastNum):

    # 將新的預測值加入參考資料(用自己的預測值往前看)
    if i > 0:
        # 將新的預測值加入 inputs
        # 將預測的發電量放入最後一個特徵，其他特徵保留原來的
        PredictValue = np.append(inputs[-1][-1, :-1], PredictOutput[i-1])  # 新預測值加入 (風速, 氣壓, 溫度, 濕度, 光照度, 預測發電量)
        NewInput = np.vstack((inputs[-1][1:], PredictValue))  # 拼接，保留最後 LookBackNum 筆
        inputs.append(NewInput)  # 確保形狀為 (12, 6)

    # 從 inputs 提取新的參考資料12筆(往前看12筆)
    X_test = np.array(inputs[-1])  # 使用最新的 inputs，形狀為 (12, 6)
    X_test = np.reshape(X_test, (1, LookBackNum, 6))  # 確保形狀為 (batch_size, timesteps, features)

    # 預測
    predicted = regressor.predict(X_test)  # 預測發電量，輸出形狀 (1, 1)
    PredictOutput.append(predicted)

# 最後檢查輸出
print("Final PredictOutput.shape:", np.array(PredictOutput).shape)

# 寫預測結果寫成新的CSV檔案
# 將預測結果轉換為 DataFrame，並寫入 CSV 檔案
df = pd.DataFrame(np.round(PredictOutput, 2).flatten())  # 展平預測結果
df.to_csv('output.csv', index=False, encoding='utf-8-sig')
print('Output CSV File Saved')


# %%

TempData.shape: (12, 6)
TempData after scaling: (12, 6)
inputs.shape: (1, 12, 6)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 368ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━